In [1]:
import numpy as np
import os.path

In [ ]:
class patchFamily2D:
    def __init__(self,file_ID):
        file_exists = os.path.exists(file_ID+'1')
        fNo = 1
        while (file_exists == True):
            f = open(file_ID+str(fNo), "r")
            data = f.read()
            data = data.split("\n")
            lenV = int(data[0])
            tempV = np.zeros(lenV)
            dataSplit = data.split("\n")
            u_knots = dataSplit[1].split(" ")
            
            
            fNo += 1
            file_exists = os.path.exists(file_ID+str(fNo))
            
            
            
